Test model dự đoán

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import PipelineModel
import pandas as pd

# Khởi tạo SparkSession
spark = SparkSession.builder.getOrCreate()

# ======= 1. Giả lập input JSON (người dùng điền quizz) =======
# Ví dụ: người dùng là nữ, sức khoẻ ổn, cao 1.6m, nặng 60kg,...
input_data = {
    "Sex": 0,
    "GeneralHealth": 3,
    "HeightInMeters": 0.0,
    "WeightInKilograms": 0.0,
    "BMI": 0.0,
    "HadHeartAttack": 0,
    "HadAngina": 0,
    "HadStroke": 0,  # Cột target này vẫn cần có để tạo schema (dù sẽ bỏ sau)
    "HadAsthma": 0,
    "HadSkinCancer": 0,
    "HadCOPD": 0,
    "HadDepressiveDisorder": 0,
    "HadKidneyDisease": 0,
    "HadArthritis": 1,
    "DeafOrHardOfHearing": 0,
    "BlindOrVisionDifficulty": 0,
    "DifficultyConcentrating": 0,
    "DifficultyWalking": 0,
    "DifficultyDressingBathing": 0,
    "DifficultyErrands": 0,
    "SmokerStatus": 1,
    "ECigaretteUsage": 0,
    "ChestScan": 1,
    "AlcoholDrinkers": 0,
    "HIVTesting": 0,
    "FluVaxLast12": 1,
    "PneumoVaxEver": 0,
    "HighRiskLastYear": 0,
    "CovidPos": 1,
    "AgeMin": 60,
    "AgeMax": 64,
    "TetanusLast10TdapIndex": 1.0,
    "HadDiabetesIndex": 1.0
}

# Chuyển dict thành Pandas rồi Spark DataFrame
input_df = pd.DataFrame([input_data])
spark_df = spark.createDataFrame(input_df)

In [ ]:
# ======= 2. Load lại mô hình từ thư mục đã lưu =======
model = PipelineModel.load("Model_LogisticRegression")  # Đường dẫn bạn đã lưu model

In [ ]:
# ======= 5. Dự đoán =======
predictions = model.transform(spark_df)

# ======= 6. Lấy kết quả dự đoán =======
result = predictions.select("probability", "prediction").collect()[0]
probability = result["probability"][1]  # Xác suất bị đột quỵ (label = 1)
prediction = result["prediction"]

# ======= 7. In ra kết quả rõ ràng =======
percent = round(probability * 100, 2)
conclusion = "Nguy cơ CAO bị đột quỵ" if probability > 0.5 else "Nguy cơ THẤP bị đột quỵ"

print(f"✅ Xác suất bị đột quỵ: {percent}%")
print(f"📌 Kết luận: {conclusion}")


✅ Xác suất bị đột quỵ: 41.42%
📌 Kết luận: Nguy cơ THẤP bị đột quỵ
